## 📊 Dataset Statistics

| Dataset | Total | Normal | Abnormal |
|---------|-------|--------|----------|
| **NIH** | 112,120 | 60,361 (53.8%) | 51,759 (46.2%) |
| **Pediatric** | 5,856 | 1,583 (27.0%) | 4,273 (73.0%) |
| **CheXpert** | 29,031 | 1,123 (3.9%) | 27,908 (96.1%) |

---
# 🔧 Setup & Environment

In [ ]:
# Check current working directory
import os
from pathlib import Path

print(f"Current working directory: {os.getcwd()}")
print(f"Python version: {os.sys.version}")

In [ ]:
# Define project paths (adjust these to your local setup)
PROJECT_ROOT = Path(r"C:\CAS AML\M3 Project")
DATA_DIR = PROJECT_ROOT / "data" / "processed"
LABELS_DIR = PROJECT_ROOT / "data" / "labels"
SCRIPTS_DIR = PROJECT_ROOT / "scripts"
RESULTS_DIR = PROJECT_ROOT / "Results"
MODELS_DIR = PROJECT_ROOT / "models"

# Create directories if they don't exist
for directory in [DATA_DIR, LABELS_DIR, SCRIPTS_DIR, RESULTS_DIR, MODELS_DIR]:
    directory.mkdir(parents=True, exist_ok=True)
    print(f"✅ {directory.name}: {directory}")

# Add scripts to path
import sys
if str(SCRIPTS_DIR) not in sys.path:
    sys.path.insert(0, str(SCRIPTS_DIR))

print(f"\n✅ Project root: {PROJECT_ROOT}")

### Import Libraries (Keras 3.x)

In [ ]:
import os
os.environ['KERAS_BACKEND'] = 'torch'  # Options: 'torch', 'tensorflow', 'jax'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.metrics import (
    roc_auc_score, accuracy_score, balanced_accuracy_score,
    classification_report, confusion_matrix, roc_curve
)
import h5py
import keras
from keras import layers, models, callbacks, optimizers
import json
from pathlib import Path
import gc

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print(f"✅ Keras version: {keras.__version__}")
print(f"✅ Keras backend: {keras.backend.backend()}")
print(f"✅ NumPy version: {np.__version__}")
print(f"✅ Pandas version: {pd.__version__}")

In [ ]:
# Check GPU availability
backend = keras.backend.backend()

if backend == 'torch':
    import torch
    print(f"PyTorch version: {torch.__version__}")
    print(f"CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"CUDA version: {torch.version.cuda}")
        print(f"GPU: {torch.cuda.get_device_name(0)}")
        print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
elif backend == 'tensorflow':
    import tensorflow as tf
    print(f"TensorFlow version: {tf.__version__}")
    print(f"GPU devices: {tf.config.list_physical_devices('GPU')}")
elif backend == 'jax':
    import jax
    print(f"JAX version: {jax.__version__}")
    print(f"JAX devices: {jax.devices()}")

---
# 📁 Phase 1: Baseline Statistical Analysis

## Phase 1a: All Data Comparison

**Goal:** Quantify baseline distribution shift using all images (mixed pathologies)

In [ ]:
# Load test images from all datasets
print("="*80)
print("PHASE 1a: LOADING TEST DATA (ALL IMAGES)")
print("="*80)

datasets = {}

for dataset_name in ['nih', 'pediatric', 'chexpert']:
    h5_path = DATA_DIR / dataset_name / 'test_all.h5'
    
    if not h5_path.exists():
        print(f"⚠️  {dataset_name}: File not found - {h5_path}")
        continue
    
    print(f"\n📂 Loading {dataset_name}...")
    
    with h5py.File(h5_path, 'r') as f:
        images = f['images'][:]
        labels = f['labels'][:]
        
    datasets[dataset_name] = {
        'images': images,
        'labels': labels,
        'n_samples': len(images)
    }
    
    print(f"   Shape: {images.shape}")
    print(f"   Samples: {len(images):,}")
    print(f"   Memory: {images.nbytes / 1024**2:.1f} MB")

print(f"\n✅ Loaded {len(datasets)} datasets")

In [ ]:
# Compute pixel statistics
print("\n" + "="*80)
print("COMPUTING PIXEL STATISTICS")
print("="*80)

stats_1a = []

for name, data in datasets.items():
    images = data['images']
    
    stat = {
        'Dataset': name.upper(),
        'Mean': images.mean(),
        'Std': images.std(),
        'Min': images.min(),
        'Max': images.max(),
        'Median': np.median(images),
        'Q25': np.percentile(images, 25),
        'Q75': np.percentile(images, 75)
    }
    stats_1a.append(stat)
    
    print(f"\n{name.upper()}:")
    print(f"  Mean:   {stat['Mean']:.4f}")
    print(f"  Std:    {stat['Std']:.4f}")
    print(f"  Range:  [{stat['Min']:.4f}, {stat['Max']:.4f}]")
    print(f"  Median: {stat['Median']:.4f}")

df_stats_1a = pd.DataFrame(stats_1a)
print("\n" + df_stats_1a.to_string(index=False))

In [ ]:
# Compute JS Divergence
from scipy.spatial.distance import jensenshannon

def compute_js_divergence(images1, images2, bins=100):
    """Compute Jensen-Shannon divergence between two image distributions"""
    hist1, _ = np.histogram(images1.flatten(), bins=bins, range=(0, 1), density=True)
    hist2, _ = np.histogram(images2.flatten(), bins=bins, range=(0, 1), density=True)
    
    # Normalize to probability distributions
    hist1 = hist1 / hist1.sum()
    hist2 = hist2 / hist2.sum()
    
    return jensenshannon(hist1, hist2)

print("\n" + "="*80)
print("COMPUTING JENSEN-SHANNON DIVERGENCE")
print("="*80)

js_results_1a = []
dataset_names = list(datasets.keys())

for i, name1 in enumerate(dataset_names):
    for name2 in dataset_names[i+1:]:
        js_div = compute_js_divergence(
            datasets[name1]['images'],
            datasets[name2]['images']
        )
        
        js_results_1a.append({
            'Dataset1': name1.upper(),
            'Dataset2': name2.upper(),
            'JS_Divergence': js_div
        })
        
        print(f"  {name1.upper():10s} vs {name2.upper():10s}: {js_div:.4f}")

df_js_1a = pd.DataFrame(js_results_1a)

# Save results
phase1a_dir = RESULTS_DIR / 'phase1a'
phase1a_dir.mkdir(parents=True, exist_ok=True)

df_stats_1a.to_csv(phase1a_dir / 'phase1a_statistics.csv', index=False)
df_js_1a.to_csv(phase1a_dir / 'phase1a_js_divergence.csv', index=False)

print(f"\n✅ Results saved to {phase1a_dir}")

In [ ]:
# Visualize JS Divergence
fig, ax = plt.subplots(figsize=(10, 6))

comparisons = [f"{row['Dataset1']} vs\n{row['Dataset2']}" for _, row in df_js_1a.iterrows()]
values = df_js_1a['JS_Divergence'].values

bars = ax.bar(comparisons, values, color=['#FF6B6B', '#4ECDC4', '#45B7D1'], alpha=0.8, edgecolor='black')

# Add value labels on bars
for bar, val in zip(bars, values):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{val:.4f}',
            ha='center', va='bottom', fontsize=12, fontweight='bold')

ax.set_ylabel('JS Divergence', fontsize=14, fontweight='bold')
ax.set_title('Phase 1a: Distribution Shift (All Images)', fontsize=16, fontweight='bold', pad=20)
ax.grid(axis='y', alpha=0.3, linestyle='--')
ax.set_ylim(0, max(values) * 1.2)

plt.tight_layout()
plt.savefig(phase1a_dir / 'phase1a_js_divergence.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Visualization saved")

## Phase 1b: Normals-Only Comparison

**Goal:** Isolate institutional/demographic shift by controlling for pathology

In [ ]:
# Load normal images only
print("="*80)
print("PHASE 1b: LOADING NORMAL IMAGES ONLY")
print("="*80)

datasets_normal = {}

for dataset_name in ['nih', 'pediatric', 'chexpert']:
    h5_path = DATA_DIR / dataset_name / 'test_normals.h5'
    
    if not h5_path.exists():
        print(f"⚠️  {dataset_name}: File not found - {h5_path}")
        continue
    
    print(f"\n📂 Loading {dataset_name} normals...")
    
    with h5py.File(h5_path, 'r') as f:
        images = f['images'][:]
        labels = f['labels'][:]
        
    datasets_normal[dataset_name] = {
        'images': images,
        'labels': labels,
        'n_samples': len(images)
    }
    
    print(f"   Shape: {images.shape}")
    print(f"   Samples: {len(images):,}")

print(f"\n✅ Loaded {len(datasets_normal)} datasets (normals only)")

In [ ]:
# Compute statistics and JS divergence for normals
print("\n" + "="*80)
print("COMPUTING STATISTICS FOR NORMALS")
print("="*80)

stats_1b = []
js_results_1b = []

# Statistics
for name, data in datasets_normal.items():
    images = data['images']
    stat = {
        'Dataset': name.upper(),
        'Mean': images.mean(),
        'Std': images.std(),
        'Min': images.min(),
        'Max': images.max(),
        'Median': np.median(images)
    }
    stats_1b.append(stat)

# JS Divergence
dataset_names = list(datasets_normal.keys())
for i, name1 in enumerate(dataset_names):
    for name2 in dataset_names[i+1:]:
        js_div = compute_js_divergence(
            datasets_normal[name1]['images'],
            datasets_normal[name2]['images']
        )
        
        js_results_1b.append({
            'Dataset1': name1.upper(),
            'Dataset2': name2.upper(),
            'JS_Divergence': js_div
        })
        
        print(f"  {name1.upper():10s} vs {name2.upper():10s}: {js_div:.4f}")

df_stats_1b = pd.DataFrame(stats_1b)
df_js_1b = pd.DataFrame(js_results_1b)

# Save results
phase1b_dir = RESULTS_DIR / 'phase1b'
phase1b_dir.mkdir(parents=True, exist_ok=True)

df_stats_1b.to_csv(phase1b_dir / 'phase1b_statistics.csv', index=False)
df_js_1b.to_csv(phase1b_dir / 'phase1b_js_divergence.csv', index=False)

print(f"\n✅ Results saved to {phase1b_dir}")

---
# 📁 Phase 2: Autoencoder Training

## Phase 2a: NIH_Full Autoencoder

**Data:** All NIH images (normals + abnormals)  
**Architecture:** Convolutional autoencoder with 256-dim latent space

In [ ]:
# Build autoencoder architecture
def build_autoencoder(input_shape=(224, 224, 1), latent_dim=256):
    """
    Build convolutional autoencoder
    
    Args:
        input_shape: Input image shape
        latent_dim: Latent space dimension
        
    Returns:
        encoder, decoder, autoencoder models
    """
    
    # ENCODER
    encoder_input = layers.Input(shape=input_shape, name='encoder_input')
    
    x = layers.Conv2D(32, 3, activation='relu', padding='same', name='enc_conv1')(encoder_input)
    x = layers.MaxPooling2D(2, padding='same', name='enc_pool1')(x)
    
    x = layers.Conv2D(64, 3, activation='relu', padding='same', name='enc_conv2')(x)
    x = layers.MaxPooling2D(2, padding='same', name='enc_pool2')(x)
    
    x = layers.Conv2D(128, 3, activation='relu', padding='same', name='enc_conv3')(x)
    x = layers.MaxPooling2D(2, padding='same', name='enc_pool3')(x)
    
    x = layers.Conv2D(256, 3, activation='relu', padding='same', name='enc_conv4')(x)
    x = layers.MaxPooling2D(2, padding='same', name='enc_pool4')(x)
    
    x = layers.Flatten(name='enc_flatten')(x)
    encoder_output = layers.Dense(latent_dim, activation='relu', name='latent')(x)
    
    encoder = models.Model(encoder_input, encoder_output, name='encoder')
    
    # DECODER
    decoder_input = layers.Input(shape=(latent_dim,), name='decoder_input')
    
    x = layers.Dense(14 * 14 * 256, activation='relu', name='dec_dense')(decoder_input)
    x = layers.Reshape((14, 14, 256), name='dec_reshape')(x)
    
    x = layers.Conv2DTranspose(128, 3, activation='relu', strides=2, padding='same', name='dec_conv1')(x)
    x = layers.Conv2DTranspose(64, 3, activation='relu', strides=2, padding='same', name='dec_conv2')(x)
    x = layers.Conv2DTranspose(32, 3, activation='relu', strides=2, padding='same', name='dec_conv3')(x)
    x = layers.Conv2DTranspose(16, 3, activation='relu', strides=2, padding='same', name='dec_conv4')(x)
    
    decoder_output = layers.Conv2D(1, 3, activation='sigmoid', padding='same', name='decoder_output')(x)
    
    decoder = models.Model(decoder_input, decoder_output, name='decoder')
    
    # AUTOENCODER
    autoencoder_input = layers.Input(shape=input_shape, name='autoencoder_input')
    encoded = encoder(autoencoder_input)
    decoded = decoder(encoded)
    autoencoder = models.Model(autoencoder_input, decoded, name='autoencoder')
    
    return encoder, decoder, autoencoder

# Build models
print("Building autoencoder architecture...")
encoder_2a, decoder_2a, autoencoder_2a = build_autoencoder()

print("\n" + "="*80)
print("ENCODER ARCHITECTURE")
print("="*80)
encoder_2a.summary()

print("\n" + "="*80)
print("DECODER ARCHITECTURE")
print("="*80)
decoder_2a.summary()

print("\n" + "="*80)
print("AUTOENCODER ARCHITECTURE")
print("="*80)
autoencoder_2a.summary()

In [ ]:
# Load NIH training data
print("="*80)
print("LOADING NIH TRAINING DATA (ALL IMAGES)")
print("="*80)

# Load train and validation
train_path = DATA_DIR / 'nih' / 'train_all.h5'
val_path = DATA_DIR / 'nih' / 'val_all.h5'

with h5py.File(train_path, 'r') as f:
    X_train = f['images'][:]
    print(f"✅ Training: {X_train.shape}")

with h5py.File(val_path, 'r') as f:
    X_val = f['images'][:]
    print(f"✅ Validation: {X_val.shape}")

# Normalize to [0, 1] if needed
if X_train.max() > 1.0:
    X_train = X_train / 255.0
    X_val = X_val / 255.0
    print("✅ Normalized to [0, 1]")

print(f"\nTraining memory: {X_train.nbytes / 1024**3:.2f} GB")
print(f"Validation memory: {X_val.nbytes / 1024**3:.2f} GB")

In [ ]:
# Compile and train autoencoder
print("="*80)
print("COMPILING AND TRAINING AUTOENCODER (PHASE 2a)")
print("="*80)

# Compile
autoencoder_2a.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

# Setup callbacks
phase2a_dir = RESULTS_DIR / 'phase2a'
phase2a_dir.mkdir(parents=True, exist_ok=True)

checkpoint_path = str(phase2a_dir / 'autoencoder_best.keras')

callbacks_list = [
    callbacks.ModelCheckpoint(
        checkpoint_path,
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    ),
    callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    )
]

# Train
print("\nStarting training...")
history_2a = autoencoder_2a.fit(
    X_train, X_train,
    epochs=100,
    batch_size=32,
    validation_data=(X_val, X_val),
    callbacks=callbacks_list,
    verbose=1
)

print("\n✅ Training complete!")

In [ ]:
# Save models and history
print("Saving models...")

autoencoder_2a.save(phase2a_dir / 'autoencoder_final.keras')
encoder_2a.save(phase2a_dir / 'encoder.keras')
decoder_2a.save(phase2a_dir / 'decoder.keras')

# Save training history
history_dict = {
    'loss': [float(x) for x in history_2a.history['loss']],
    'val_loss': [float(x) for x in history_2a.history['val_loss']],
    'mae': [float(x) for x in history_2a.history.get('mae', [])],
    'val_mae': [float(x) for x in history_2a.history.get('val_mae', [])]
}

with open(phase2a_dir / 'training_history.json', 'w') as f:
    json.dump(history_dict, f, indent=2)

# Save metadata
metadata = {
    'dataset': 'NIH_Full',
    'train_samples': len(X_train),
    'val_samples': len(X_val),
    'epochs_trained': len(history_2a.history['loss']),
    'best_val_loss': float(min(history_2a.history['val_loss'])),
    'architecture': 'ConvAutoencoder',
    'latent_dim': 256,
    'backend': keras.backend.backend()
}

with open(phase2a_dir / 'metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"✅ All files saved to {phase2a_dir}")

In [ ]:
# Plot training curves
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Loss
axes[0].plot(history_2a.history['loss'], label='Train Loss', linewidth=2)
axes[0].plot(history_2a.history['val_loss'], label='Val Loss', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('MSE Loss', fontsize=12)
axes[0].set_title('Phase 2a: Training Loss', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# MAE
if 'mae' in history_2a.history:
    axes[1].plot(history_2a.history['mae'], label='Train MAE', linewidth=2)
    axes[1].plot(history_2a.history['val_mae'], label='Val MAE', linewidth=2)
    axes[1].set_xlabel('Epoch', fontsize=12)
    axes[1].set_ylabel('MAE', fontsize=12)
    axes[1].set_title('Phase 2a: Mean Absolute Error', fontsize=14, fontweight='bold')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(phase2a_dir / 'training_curves.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Training curves saved")

## Phase 2b: NIH_Normal Autoencoder

**Data:** Only normal NIH images  
**Purpose:** Learn "normal" appearance to detect distribution shift

In [ ]:
# Build and train Phase 2b autoencoder (normals only)
# [Similar code to Phase 2a, but using *_normals.h5 files]

print("="*80)
print("PHASE 2b: NIH_NORMAL AUTOENCODER")
print("="*80)

# Load normal images only
train_normal_path = DATA_DIR / 'nih' / 'train_normals.h5'
val_normal_path = DATA_DIR / 'nih' / 'val_normals.h5'

with h5py.File(train_normal_path, 'r') as f:
    X_train_normal = f['images'][:]
    print(f"✅ Training (normals): {X_train_normal.shape}")

with h5py.File(val_normal_path, 'r') as f:
    X_val_normal = f['images'][:]
    print(f"✅ Validation (normals): {X_val_normal.shape}")

# Normalize
if X_train_normal.max() > 1.0:
    X_train_normal = X_train_normal / 255.0
    X_val_normal = X_val_normal / 255.0

# Build new autoencoder
encoder_2b, decoder_2b, autoencoder_2b = build_autoencoder()

# Compile
autoencoder_2b.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

# Setup callbacks
phase2b_dir = RESULTS_DIR / 'phase2b'
phase2b_dir.mkdir(parents=True, exist_ok=True)

callbacks_2b = [
    callbacks.ModelCheckpoint(
        str(phase2b_dir / 'autoencoder_best.keras'),
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    ),
    callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    )
]

# Train
print("\nStarting training (normals only)...")
history_2b = autoencoder_2b.fit(
    X_train_normal, X_train_normal,
    epochs=100,
    batch_size=32,
    validation_data=(X_val_normal, X_val_normal),
    callbacks=callbacks_2b,
    verbose=1
)

# Save models
autoencoder_2b.save(phase2b_dir / 'autoencoder_final.keras')
encoder_2b.save(phase2b_dir / 'encoder.keras')
decoder_2b.save(phase2b_dir / 'decoder.keras')

print("\n✅ Phase 2b complete!")

---
# 📁 Phase 3: Reconstruction Error Analysis

## Phase 3a: NIH_Full Autoencoder on All Test Images

In [ ]:
# Compute reconstruction errors
print("="*80)
print("PHASE 3a: RECONSTRUCTION ERROR ANALYSIS")
print("="*80)

# Load best autoencoder
autoencoder_2a_best = keras.models.load_model(phase2a_dir / 'autoencoder_best.keras')

results_3a = {}

for dataset_name in ['nih', 'pediatric', 'chexpert']:
    print(f"\n📊 Processing {dataset_name.upper()}...")
    
    # Load test images
    test_path = DATA_DIR / dataset_name / 'test_all.h5'
    
    with h5py.File(test_path, 'r') as f:
        X_test = f['images'][:]
    
    if X_test.max() > 1.0:
        X_test = X_test / 255.0
    
    # Reconstruct
    X_reconstructed = autoencoder_2a_best.predict(X_test, batch_size=32, verbose=0)
    
    # Compute per-image MSE
    errors = np.mean((X_test - X_reconstructed) ** 2, axis=(1, 2, 3))
    
    results_3a[dataset_name] = {
        'mean': float(errors.mean()),
        'std': float(errors.std()),
        'median': float(np.median(errors)),
        'min': float(errors.min()),
        'max': float(errors.max()),
        'errors': errors
    }
    
    print(f"   Mean error: {errors.mean():.6f}")
    print(f"   Std:        {errors.std():.6f}")
    print(f"   Range:      [{errors.min():.6f}, {errors.max():.6f}]")

# Save results
phase3a_dir = RESULTS_DIR / 'phase3a'
phase3a_dir.mkdir(parents=True, exist_ok=True)

summary_3a = pd.DataFrame([
    {
        'Dataset': name.upper(),
        'Mean_Error': results_3a[name]['mean'],
        'Std_Error': results_3a[name]['std'],
        'Median_Error': results_3a[name]['median']
    }
    for name in results_3a.keys()
])

summary_3a.to_csv(phase3a_dir / 'phase3a_summary.csv', index=False)

with open(phase3a_dir / 'phase3a_statistics.json', 'w') as f:
    json.dump({k: {kk: vv for kk, vv in v.items() if kk != 'errors'} 
               for k, v in results_3a.items()}, f, indent=2)

print(f"\n✅ Results saved to {phase3a_dir}")

In [ ]:
# Visualize reconstruction errors
fig, ax = plt.subplots(figsize=(10, 6))

dataset_order = ['nih', 'pediatric', 'chexpert']
means = [results_3a[d]['mean'] for d in dataset_order]
labels = [d.upper() for d in dataset_order]

bars = ax.bar(labels, means, color=['#4ECDC4', '#FF6B6B', '#45B7D1'], alpha=0.8, edgecolor='black')

for bar, val in zip(bars, means):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{val:.6f}',
            ha='center', va='bottom', fontsize=11, fontweight='bold')

ax.set_ylabel('Mean Reconstruction Error (MSE)', fontsize=14, fontweight='bold')
ax.set_title('Phase 3a: Reconstruction Error by Dataset', fontsize=16, fontweight='bold', pad=20)
ax.grid(axis='y', alpha=0.3, linestyle='--')

plt.tight_layout()
plt.savefig(phase3a_dir / 'phase3a_reconstruction_errors.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Visualization saved")

---
# 📁 Phase 4: Classifier Training & Evaluation

**Model:** DenseNet121-based binary classifier (Normal vs Abnormal)

In [ ]:
# Build classifier (DenseNet121 base)
from keras.applications import DenseNet121

def build_classifier(input_shape=(224, 224, 3)):
    """
    Build binary classifier using DenseNet121 backbone
    """
    base_model = DenseNet121(
        include_top=False,
        weights='imagenet',
        input_shape=input_shape,
        pooling='avg'
    )
    
    # Freeze early layers
    for layer in base_model.layers[:100]:
        layer.trainable = False
    
    # Build classifier head
    inputs = layers.Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    model = models.Model(inputs, outputs, name='chest_xray_classifier')
    
    return model

print("Building classifier...")
classifier = build_classifier()
classifier.summary()

In [ ]:
# Load and prepare NIH data for classifier training
print("="*80)
print("LOADING NIH DATA FOR CLASSIFIER TRAINING")
print("="*80)

# Load train/val/test
with h5py.File(DATA_DIR / 'nih' / 'train_all.h5', 'r') as f:
    X_train_clf = f['images'][:]
    y_train_clf = f['labels'][:, 0]  # Column 0 = 'No Finding' (1=normal, 0=abnormal)
    # Invert labels: 0=normal, 1=abnormal
    y_train_clf = 1 - y_train_clf

with h5py.File(DATA_DIR / 'nih' / 'val_all.h5', 'r') as f:
    X_val_clf = f['images'][:]
    y_val_clf = 1 - f['labels'][:, 0]

with h5py.File(DATA_DIR / 'nih' / 'test_all.h5', 'r') as f:
    X_test_clf = f['images'][:]
    y_test_clf = 1 - f['labels'][:, 0]

# Normalize
if X_train_clf.max() > 1.0:
    X_train_clf = X_train_clf / 255.0
    X_val_clf = X_val_clf / 255.0
    X_test_clf = X_test_clf / 255.0

# Convert grayscale to RGB (DenseNet expects 3 channels)
X_train_clf = np.repeat(X_train_clf, 3, axis=-1)
X_val_clf = np.repeat(X_val_clf, 3, axis=-1)
X_test_clf = np.repeat(X_test_clf, 3, axis=-1)

print(f"✅ Training: {X_train_clf.shape}, Labels: {y_train_clf.shape}")
print(f"✅ Validation: {X_val_clf.shape}, Labels: {y_val_clf.shape}")
print(f"✅ Test: {X_test_clf.shape}, Labels: {y_test_clf.shape}")
print(f"\nClass distribution (train): {np.bincount(y_train_clf.astype(int))}")

In [ ]:
# Compile and train classifier
print("="*80)
print("TRAINING CLASSIFIER")
print("="*80)

# Compute class weights for imbalanced data
from sklearn.utils.class_weight import compute_class_weight

class_weights_array = compute_class_weight(
    'balanced',
    classes=np.unique(y_train_clf),
    y=y_train_clf
)
class_weights = {i: class_weights_array[i] for i in range(len(class_weights_array))}

print(f"Class weights: {class_weights}")

# Compile
classifier.compile(
    optimizer=optimizers.Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy', keras.metrics.AUC(name='auc')]
)

# Setup callbacks
phase4_dir = RESULTS_DIR / 'phase4'
phase4_dir.mkdir(parents=True, exist_ok=True)

callbacks_4 = [
    callbacks.ModelCheckpoint(
        str(phase4_dir / 'classifier_best.keras'),
        monitor='val_auc',
        mode='max',
        save_best_only=True,
        verbose=1
    ),
    callbacks.EarlyStopping(
        monitor='val_auc',
        mode='max',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    )
]

# Train
print("\nStarting training...")
history_4 = classifier.fit(
    X_train_clf, y_train_clf,
    epochs=50,
    batch_size=32,
    validation_data=(X_val_clf, y_val_clf),
    class_weight=class_weights,
    callbacks=callbacks_4,
    verbose=1
)

print("\n✅ Classifier training complete!")

---
# 📁 Phase 5: Correlation Analysis

**Goal:** Test if reconstruction error predicts classifier performance degradation

In [ ]:
# Compute correlations between reconstruction error and classifier performance
print("="*80)
print("PHASE 5: CORRELATION ANALYSIS")
print("="*80)

# Load Phase 3 and Phase 4 results
# Combine reconstruction errors with classifier metrics
# Compute Pearson and Spearman correlations

phase5_dir = RESULTS_DIR / 'phase5'
phase5_dir.mkdir(parents=True, exist_ok=True)

correlation_results = {
    'reconstruction_error': [],
    'balanced_accuracy': [],
    'auc': []
}

# [Add correlation computation code here]

print("\n✅ Correlation analysis complete!")
print(f"Results saved to {phase5_dir}")

---
# 📊 Summary & Conclusions

All phases complete! Check the `Results/` directory for:
- Phase 1: Statistical analysis and JS divergence
- Phase 2: Trained autoencoders
- Phase 3: Reconstruction error analysis
- Phase 4: Classifier evaluation
- Phase 5: Correlation results